### Code pour récupérer la base de données de vents

On va extraire l'ensemble des vents pertinents pour les éoliennes en croisant leurs coordonnées avec celles des vents de météo-france.

In [43]:
#On récupère les éoliennes
df = pd.read_csv("/home/onyxia/work/ProjPython2A/eolien.csv", sep=";")

Nous convertissons nos données afin qu'elles soient exploitables par geopandas

On crée la variable geometry qui nous permettra de manipuler la position des éoliennes plus simplement.

In [60]:
geometrie = [Point(xy) for xy in zip(df.x_aerogenerateur,df.y_aerogenerateur)] # colonnes du DataFrame résultants
caract_eoliennes = gpd.GeoDataFrame(df,geometry=geometrie)
caract_eoliennes = caract_eoliennes.set_crs(2154, allow_override=True)

In [62]:
#Le code est long à tourner et les données sont déjà fournies!



caract_eoliennes = caract_eoliennes.to_crs(4326)

base_url = "https://archive-api.open-meteo.com/v1/era5?"

end_date="2021-01-02"
end_date_comparaison = pd.to_datetime(end_date)
fichier_csv = open(r'/home/onyxia/work/ProjPython2A/donnees_vent.csv','w',newline='')

writer = csv.writer(fichier_csv)

for eolienne in caract_eoliennes.itertuples():
    a = eolienne.geometry
    longitude=str(a.x)
    latitude=str(a.y)
    date = pd.to_datetime(eolienne.date_mise_en_service).date()
    if pd.isnull(date):
        start_date = "2021-01-01" 
    elif date < end_date_comparaison:
        start_date=str(max(date, pd.to_datetime("2021-01-01").date()))
        complete_url = base_url + "latitude=" + latitude + "&longitude=" + longitude + "&hourly=windspeed_10m&windspeed_unit=ms&start_date="+start_date+"&end_date="+end_date
        reponse = requests.get(complete_url)
        donnees_vent_eolienne = pd.DataFrame(reponse.json())['hourly']
        writer.writerow([eolienne.Index,donnees_vent_eolienne['time'],donnees_vent_eolienne['windspeed_10m']])
    if eolienne.Index%50 == 0 : 
            print(eolienne.Index)


/tmp/ipykernel_162/780661846.py:22: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  elif date < end_date_comparaison:


0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100


KeyboardInterrupt: 

In [ ]:
caract_eoliennes.head()

Ici on réalise le même code que précédemment mais pour obtenir la météo prédite


In [63]:
#Le code est long à tourner

import csv

caract_eoliennes = caract_eoliennes.to_crs(4326)

base_url = "https://api.open-meteo.com/v1/forecast?"

end_date_comparaison = pd.to_datetime(end_date)
fichier_csv = open(r'/home/onyxia/work/ProjPython2A/donnees_vent_prediction.csv','w',newline='')

writer = csv.writer(fichier_csv)


start_date = "2022-12-13"

end_date = "2022-12-15"


for eolienne in caract_eoliennes.itertuples():
    a = eolienne.geometry
    longitude=str(a.x)
    latitude=str(a.y)
    date = pd.to_datetime(eolienne.date_mise_en_service).date()
    complete_url = base_url + "latitude=" + latitude + "&longitude=" + longitude + "&hourly=windspeed_10m&windspeed_unit=ms&start_date="+start_date+"&end_date="+end_date
    reponse = requests.get(complete_url)
    donnees_vent_eolienne = pd.DataFrame(reponse.json())['hourly']
    writer.writerow([eolienne.Index,donnees_vent_eolienne['time'],donnees_vent_eolienne['windspeed_10m']])
    if eolienne.Index%50 == 0 : 
            print(eolienne.Index)

0
50


KeyboardInterrupt: 